In [1]:
%load_ext autoreload

%autoreload 1

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [28]:
import os
from pprint import pprint

In [29]:
from akd.config import CONFIG

In [44]:
# CONFIG

# Import

In [6]:
from akd.tools.search import (
    SearxNGSearchTool,
    SearxNGSearchToolConfig,
    SearxNGSearchToolInputSchema
)

In [7]:
from akd.tools.scrapers.web_scrapers import (
    SimpleWebScraper,
    WebpageScraperToolConfig,
    WebpageScraperToolInputSchema,
    Crawl4AIWebScraper
)
from akd.tools.scrapers.pdf_scrapers import (
    SimplePDFScraper,
)
from akd.tools.scrapers.composite import CompositeWebScraper

In [8]:
from akd.tools.scrapers.resolvers import (
    ArxivResolver,
    ADSResolver,
    IdentityResolver,
)

from akd.tools.scrapers.composite import ResearchArticleResolver

In [9]:
from akd.agents.factory import (
    create_intent_agent,
    create_extraction_agent,
    create_intent_agent,
    create_query_agent
)
from akd.agents.extraction import IntentBasedExtractionSchemaMapper

In [10]:
from akd.agents.litsearch import LitAgent

# Build Tools and Agents

In [13]:
SEARCH_TOOL = SearxNGSearchTool(
    config=SearxNGSearchToolConfig(
        base_url="http://localhost:8080",
        max_results=5,
        engines=["google", "arxiv", "google_scholar"],
        debug=False
    )
)

In [14]:
scraper_cfg = WebpageScraperToolConfig()
SCRAPER = CompositeWebScraper(
    SimpleWebScraper(scraper_cfg),
    Crawl4AIWebScraper(scraper_cfg),
    SimplePDFScraper(scraper_cfg),
    debug=True
)

In [15]:
ARTICLE_RESOLVER = ResearchArticleResolver(
    ArxivResolver(),
    ADSResolver(),
    IdentityResolver()
)

In [16]:
# ARTICLE_RESOLVER.run("https://arxiv.org")

In [17]:
INTENT_AGENT = create_intent_agent()

In [18]:
QUERY_AGENT = create_query_agent()

In [19]:
lit_agent = LitAgent(
    intent_agent=INTENT_AGENT,
    schema_mapper=IntentBasedExtractionSchemaMapper(),
    query_agent=QUERY_AGENT,
    search_tool=SEARCH_TOOL,
    web_scraper=SCRAPER,
    article_resolver=ARTICLE_RESOLVER,
)
lit_agent.clear_history()

2025-03-20 10:31:01.531 | WARNING  | akd.agents.litsearch:clear_history:106 - Clearing history for all the agents


In [20]:
# query = "mangrove forest statuses in cameroon"
# query = "flash flood in south east asia land cover"
query = "methods and estimation to map landslides in Nepal"

In [21]:
result = lit_agent.run(query)

2025-03-20 10:31:03.382 | DEBUG    | akd.agents.litsearch:run:43 - query=methods and estimation to map landslides in Nepal | intent=Intent.ESTIMATION
2025-03-20 10:31:03.383 | DEBUG    | akd.agents.litsearch:run:46 - Extraction schema=typing.List[akd.structures.SingleEstimation]
2025-03-20 10:31:03.408 | INFO     | akd.agents.litsearch:run:50 - Analyzing input query to generate relevant search queries...
2025-03-20 10:31:05.010 | DEBUG    | akd.agents.litsearch:run:55 - Generated search queries:
2025-03-20 10:31:05.012 | DEBUG    | akd.agents.litsearch:run:57 - Query 1: methods and estimation to map landslides in Nepal
2025-03-20 10:31:05.013 | DEBUG    | akd.agents.litsearch:run:57 - Query 2: landslide mapping methods Nepal
2025-03-20 10:31:05.014 | DEBUG    | akd.agents.litsearch:run:57 - Query 3: landslide estimation techniques Nepal
2025-03-20 10:31:05.015 | DEBUG    | akd.agents.litsearch:run:57 - Query 4: remote sensing landslide detection Nepal
2025-03-20 10:31:05.016 | INFO    

[INIT].... → Crawl4AI 0.5.0.post4
[FETCH]... ↓ https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Status: True | Time: 4.80s
[SCRAPE].. ◆ https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Time: 0.014s
[COMPLETE] ● https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-... | Status: True | Total: 4.82s


2025-03-20 10:31:20.355 | DEBUG    | akd.agents.litsearch:run:86 - Result 5: Landslide susceptibility mapping using the weight of evidence method in the Tinau watershed, Nepal | https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-4abd-929e-79249a9b579a |   * [Skip to main navigation](https://hdl.handle.net/20.500.14017/8df80ddb-1fa4-4abd-929e-79249a9b57.. | words=1265
2025-03-20 10:31:30.818 | DEBUG    | akd.agents.litsearch:run:98 - Source=https://link.springer.com/article/10.1007/s12665-021-09650-2 | Answer=[SingleEstimation(answer='The study conducted landslide susceptibility mapping (LSM) in the Indrawati watershed, Central Nepal, using remote sensing data and four different methods: frequency ratio, logistic regression, artificial neural network, and support vector machine. The artificial neural network method showed the best prediction capability with an AUC value of 86.9%.', related_knowledge=['Landslide susceptibility mapping (LSM) is crucial for identifying areas prone to landsl

In [24]:
len(result)

5

In [25]:
result

[ExtractionDTO(source='https://link.springer.com/article/10.1007/s12665-021-09650-2', result=[SingleEstimation(answer='The study conducted landslide susceptibility mapping (LSM) in the Indrawati watershed, Central Nepal, using remote sensing data and four different methods: frequency ratio, logistic regression, artificial neural network, and support vector machine. The artificial neural network method showed the best prediction capability with an AUC value of 86.9%.', related_knowledge=['Landslide susceptibility mapping (LSM) is crucial for identifying areas prone to landslides and implementing preventive measures.', 'The study area is the Indrawati watershed in Central Nepal, a high mountain region.', 'Four methods were compared: frequency ratio, logistic regression, artificial neural network, and support vector machine.', 'Artificial neural network showed the highest prediction capability with an AUC of 86.9%.'], research_data=ResearchData(data_format='Satellite images', origin='On-s

In [42]:
# pprint(result[0])

In [43]:
# pprint(result[0].result)

# Display result

In [34]:
from typing import List, TypeVar, Any
from pydantic import BaseModel
import json
from pprint import pformat

T = TypeVar('T', bound=BaseModel)

In [36]:
def display_result(models_list: List[T], indent: int = 2) -> None:
    """
    Alternative version using model_dump_json method directly.
    
    Args:
        models_list: A list of Pydantic model instances
        indent: Number of spaces for indentation (default: 2)
    """
    if not models_list:
        print("[]")
        return
    
    print(f"[{len(models_list)} items]")
    
    for i, model in enumerate(models_list):
        # Use Pydantic's built-in JSON serialization
        formatted_model = model.model_dump_json(indent=indent)
        
        print(f"\nItem {i + 1}:")
        print(formatted_model)

In [41]:
for res in result:
    print(f"{res.source}")
    display_result(res.result)
    print("-"*42)

https://link.springer.com/article/10.1007/s12665-021-09650-2
[1 items]

Item 1:
{
  "answer": "The study conducted landslide susceptibility mapping (LSM) in the Indrawati watershed, Central Nepal, using remote sensing data and four different methods: frequency ratio, logistic regression, artificial neural network, and support vector machine. The artificial neural network method showed the best prediction capability with an AUC value of 86.9%.",
  "related_knowledge": [
    "Landslide susceptibility mapping (LSM) is crucial for identifying areas prone to landslides and implementing preventive measures.",
    "The study area is the Indrawati watershed in Central Nepal, a high mountain region.",
    "Four methods were compared: frequency ratio, logistic regression, artificial neural network, and support vector machine.",
    "Artificial neural network showed the highest prediction capability with an AUC of 86.9%."
  ],
  "research_data": {
    "data_format": "Satellite images",
    "origi